In [ ]:
from mendeleev.fetch import fetch_table
import numpy as np


periodic_table = fetch_table('elements')
feats = ['atomic_number', 'atomic_radius', 'atomic_volume', 'group_id', 'period', 'en_pauling']
atom_feat_table = periodic_table[feats].values



In [ ]:
def strucutre_to_graph(structure, r=3):
    neighbors = structure.get_all_neighbors(r=r)
    pairs = []
    atoms = []
    edge_index = []
    edge_attr = []
    for i, nns in enumerate(neighbors):
        atoms.append(structure[i].specie.number - 1)
        for n in nns:
            pair = (i, n.index, f'{n.nn_distance:.8f}')
            if pair not in pairs:
                pairs.append(pair)
                edge_index.append(pair[:2])
                edge_attr.append(float(pair[2]))
    x = atom_feat_table[atoms]
    return {'x':x, 'edge_index':np.array(edge_index), 'edge_attr':np.array(edge_attr)}

## CGConv